## DC House Price

#### Project goal: 

- Predict the median house price in each region?
- Predict the median house price in each regroup region?
- Clustering method: group the regions
- Classfication: use the right housing statistics
- Modeling: time series linear regression 
- 

In [35]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [36]:
time_frame = pd.date_range('2012-02-01','2019-10-01', 
              freq='MS').strftime("%Y-%b").tolist()

In [37]:
df_temp = pd.DataFrame(time_frame, columns = ['Time'])

In [38]:
df = pd.read_csv("../data/data_crosstab_condo.csv",  encoding='utf-16', sep='\t' )
# reformat the sale price from string to float

df_temp["Median Sale Price"] = df["Median Sale Price"].str.replace(",", "").str.replace("$", "").str.replace("K","000").astype(float)
df_temp["Median Sale Price MoM"] = df["Median Sale Price MoM "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)
df_temp["Median Sale Price YoY"] = df["Median Sale Price YoY "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)

df_temp["Homes Sold"] = df["Homes Sold"].str.replace(",", "").str.replace("$", "").str.replace("K","000").astype(float)
df_temp["Homes Sold MoM"] = df["Homes Sold MoM "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)
df_temp["Homes Sold YoY"] = df["Homes Sold YoY "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)

df_temp["New Listings"] = df["New Listings"].str.replace(",", "").str.replace("$", "").str.replace("K","000").astype(float)
df_temp["New Listings MoM"] = df["New Listings MoM "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)
df_temp["New Listings YoY"] = df["New Listings YoY "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)

df_temp["Inventory"] = df["Inventory"].str.replace(",", "").str.replace("$", "").str.replace("K","000").astype(float)
df_temp["Inventory MoM"] = df["Inventory MoM "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)
df_temp["Inventory YoY"] = df[" Inventory YoY "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)

df_temp["Days on Market"] = df["Days on Market"]
df_temp["Days on Market MoM"] = df["Days on Market MoM"]
df_temp["Days on Market YoY"] = df["Days on Market YoY"]


#df_temp["Days on Market"] = df["Days on Market"].str.replace(",", "").str.replace("$", "").str.replace("K","000").astype(float)
#df_temp["Days on Market MoM"] = df["Days on Market MoM"].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)
#df_temp["Days on Market YoY"] = df["Days on Market YoY"].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)



df_temp["Average Sale To List%"] = df["Average Sale To List"].str.replace(",", "").str.replace("%", "").str.replace("K","000").astype(float)
df_temp["Average Sale To List MoM"] = df["Average Sale To List MoM "].str.replace(",", "").str.replace("%", "").str.replace("K","000").astype(float)
df_temp["Average Sale To List YoY"] = df["Average Sale To List YoY "].str.replace(",", "").str.replace("$", "").str.replace("%", "").str.replace("K","000").astype(float)



In [39]:
df.head()

,Region,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,New Listings MoM,New Listings YoY,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY
0,"Washington, DC metro area",January 2010,$225K,NaN,NaN,542,NaN,NaN,"1,110",NaN,NaN,"3,142",NaN,NaN,56,NaN,NaN,97.8%,NaN,NaN
1,"Washington, DC metro area",February 2010,$250K,11.1%,NaN,521,-3.9%,NaN,"1,085",-2.3%,NaN,"3,346",6.5%,NaN,51,-5.0,NaN,98.2%,0.4%,NaN
2,"Washington, DC metro area",March 2010,$237K,-5.2%,NaN,786,50.9%,NaN,"1,681",54.9%,NaN,"3,670",9.7%,NaN,42,-9.0,NaN,98.2%,0.0%,NaN
3,"Washington, DC metro area",April 2010,$260K,9.7%,NaN,909,15.6%,NaN,"1,719",2.3%,NaN,"3,892",6.0%,NaN,28,-14.0,NaN,98.1%,-0.1%,NaN
4,"Washington, DC metro area",May 2010,$250K,-3.8%,NaN,988,8.7%,NaN,"1,029",-40.1%,NaN,"3,874",-0.5%,NaN,30,2.0,NaN,98.0%,-0.1%,NaN


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 20 columns):
Region                       119 non-null object
Month of Period End          119 non-null object
Median Sale Price            119 non-null object
Median Sale Price MoM        118 non-null object
Median Sale Price YoY        107 non-null object
Homes Sold                   119 non-null object
Homes Sold MoM               118 non-null object
Homes Sold YoY               107 non-null object
New Listings                 119 non-null object
New Listings MoM             118 non-null object
New Listings YoY             107 non-null object
Inventory                    119 non-null object
Inventory MoM                118 non-null object
 Inventory YoY               107 non-null object
Days on Market               119 non-null int64
Days on Market MoM           118 non-null float64
Days on Market YoY           107 non-null float64
Average Sale To List         119 non-null object
Average Sale

In [33]:
df.columns

Index(['Region', 'Month of Period End', 'Median Sale Price',
       'Median Sale Price MoM ', 'Median Sale Price YoY ', 'Homes Sold',
       'Homes Sold MoM ', 'Homes Sold YoY ', 'New Listings',
       'New Listings MoM ', 'New Listings YoY ', 'Inventory', 'Inventory MoM ',
       ' Inventory YoY ', 'Days on Market', 'Days on Market MoM',
       'Days on Market YoY', 'Average Sale To List',
       'Average Sale To List MoM ', 'Average Sale To List YoY '],
      dtype='object')

In [34]:
df_temp

,Time,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,New Listings MoM,New Listings YoY,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY,Average Sale To List%
0,2012-Feb,225000.0,NaN,NaN,542.0,NaN,NaN,1110.0,NaN,NaN,3142.0,NaN,NaN,56,NaN,NaN,97.8,NaN,NaN,97.8
1,2012-Mar,250000.0,11.1,NaN,521.0,-3.9,NaN,1085.0,-2.3,NaN,3346.0,6.5,NaN,51,-5.0,NaN,98.2,0.4,NaN,98.2
2,2012-Apr,237000.0,-5.2,NaN,786.0,50.9,NaN,1681.0,54.9,NaN,3670.0,9.7,NaN,42,-9.0,NaN,98.2,0.0,NaN,98.2
3,2012-May,260000.0,9.7,NaN,909.0,15.6,NaN,1719.0,2.3,NaN,3892.0,6.0,NaN,28,-14.0,NaN,98.1,-0.1,NaN,98.1
4,2012-Jun,250000.0,-3.8,NaN,988.0,8.7,NaN,1029.0,-40.1,NaN,3874.0,-0.5,NaN,30,2.0,NaN,98.0,-0.1,NaN,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2019-Jun,303000.0,6.1,0.8,1209.0,20.8,7.1,1482.0,3.6,7.5,2212.0,3.6,-19.5,16,-4.0,-9.0,99.1,0.1,-0.1,99.1
89,2019-Jul,294000.0,-2.8,-0.4,1253.0,3.6,6.8,1458.0,-1.6,8.6,2246.0,1.5,-16.8,19,3.0,-7.0,99.1,-0.1,0.1,99.1
90,2019-Aug,315000.0,7.2,6.8,1163.0,-7.2,11.1,1137.0,-22.0,2.5,2163.0,-3.7,-13.5,21,2.0,-12.0,99.3,0.2,0.5,99.3
91,2019-Sep,289000.0,-8.3,0.0,1101.0,-5.3,3.5,1136.0,-0.1,12.9,2044.0,-5.5,-12.0,27,6.0,-6.0,99.1,-0.1,0.5,99.1


In [6]:

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

import statsmodels.api as sm

#sm.stats.acorr_ljungbox

#import statsmodels.stats.diagnostic.acorr_breusch_godfrey as bg_test
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox
#import statsmodels.stats.diagnostic.acorr_ljungbox as ljb_test

from statsmodels.tsa.vector_ar.vecm import coint_johansen 

In [ ]:
#ACF plot

sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40)
sm.graphics.tsa.plot_pacf(dta.values.squeeze(), lags=40)